# Portfolio Analysis

This notebook implements a portfolio optimization strategy using the riskfolio library. It includes data downloading, portfolio optimization, and performance analysis.

In [10]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import riskfolio as rp

from app.utils import download_data


# Logging setup
log_dir = "logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logging.basicConfig(
    filename=os.path.join(log_dir, "portfolio_analysis.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Configuration
YEARS = 4.5  # Set timeframe in years for daily data
TICKERS = ["BTC-USD", "SPY"]

## Data Download and Preprocessing

In [11]:
def download_and_preprocess_data(tickers, years):
    data = {}
    for ticker in tickers:
        try:
            df = download_data(ticker, False, years)
            if isinstance(df, pl.DataFrame):
                # Convert Polars DataFrame to Pandas DataFrame
                df = df.to_pandas()
            # Ensure the index is datetime
            if not isinstance(df.index, pd.DatetimeIndex):
                df.index = pd.to_datetime(df.index)
            data[ticker] = df["close"]  # Assuming 'close' is the column we want
        except Exception as e:
            logging.error(f"Error downloading data for {ticker}: {e!s}")
            print(f"Error downloading data for {ticker}. Skipping.")

    # Combine data
    combined_data = pd.concat(data.values(), axis=1, keys=data.keys())

    # Ensure the index is sorted
    combined_data = combined_data.sort_index()

    # Calculate returns
    returns = combined_data.pct_change(fill_method=None).dropna()

    return returns


returns = download_and_preprocess_data(TICKERS, YEARS)
print(returns.head())

[*********************100%***********************]  1 of 1 completed


Error downloading data for BTC-USD. Skipping.


[*********************100%***********************]  1 of 1 completed

Error downloading data for SPY. Skipping.


ValueError: No objects to concatenate

## Portfolio Optimization

In [ ]:
def optimize_portfolio(returns):
    # Create portfolio object
    port = rp.Portfolio(returns)

    # Set up portfolio parameters
    method_mu = "hist"  # Method to estimate expected returns
    method_cov = "hist"  # Method to estimate covariance matrix

    # Estimate inputs
    port.assets_stats(method_mu=method_mu, method_cov=method_cov)

    # Set up optimization parameters
    model = "Classic"  # Markowitz model
    rm = "MV"  # Risk measure: Variance
    obj = "Sharpe"  # Objective: Maximize Sharpe ratio
    hist = True  # Use historical scenarios
    rf = 0  # Risk-free rate
    l = 0  # Risk aversion factor

    # Add constraints
    constraints = {
        "Disabled": False,  # Enable constraints
        "Type": "Exact",
        "Sum": 1,  # Sum of weights must be 1
        "LowerBound": 0.05,  # Minimum weight per asset
        "UpperBound": 0.4,  # Maximum weight per asset
    }

    # Optimize portfolio
    w = port.optimization(
        model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist, constraints=constraints
    )

    return w


weights = optimize_portfolio(returns)
print("Optimal portfolio weights:")
print(weights.T)

## Performance Analysis

In [ ]:
def calculate_performance(returns, weights):
    portfolio_returns = (returns * weights.T.values).sum(axis=1)

    annualized_return = portfolio_returns.mean() * 252
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = (
        annualized_return / annualized_volatility if annualized_volatility != 0 else 0
    )

    print("\nPortfolio Metrics:")
    print(f"Annualized Return: {annualized_return:.2%}")
    print(f"Annualized Volatility: {annualized_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    for asset in returns.columns:
        asset_return = returns[asset]
        asset_weight = weights.T[asset].values[0]
        asset_annualized_return = asset_return.mean() * 252
        asset_annualized_volatility = asset_return.std() * np.sqrt(252)
        asset_sharpe_ratio = (
            asset_annualized_return / asset_annualized_volatility
            if asset_annualized_volatility != 0
            else 0
        )

        print(f"\n{asset} Metrics:")
        print(f"Weight: {asset_weight:.2%}")
        print(f"Annualized Return: {asset_annualized_return:.2%}")
        print(f"Annualized Volatility: {asset_annualized_volatility:.2%}")
        print(f"Sharpe Ratio: {asset_sharpe_ratio:.2f}")

    return portfolio_returns


portfolio_returns = calculate_performance(returns, weights)

## Visualization

In [ ]:
def visualize_portfolio(weights, portfolio_returns):
    # Pie chart of asset allocation
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.pie(weights.T.values[0], labels=weights.T.columns, autopct="%1.1f%%")
    plt.title("Portfolio Allocation")

    # Cumulative returns plot
    plt.subplot(1, 2, 2)
    (1 + portfolio_returns).cumprod().plot()
    plt.title("Cumulative Portfolio Returns")
    plt.ylabel("Cumulative Returns")

    plt.tight_layout()
    plt.show()


visualize_portfolio(weights, portfolio_returns)

## Rolling Window Analysis

In [ ]:
def rolling_window_analysis(returns, window=252):
    rolling_sharpe = returns.rolling(window).apply(
        lambda x: np.sqrt(252) * x.mean() / x.std()
    )

    plt.figure(figsize=(12, 6))
    rolling_sharpe.plot()
    plt.title(f"Rolling {window}-day Sharpe Ratio")
    plt.ylabel("Sharpe Ratio")
    plt.show()


rolling_window_analysis(portfolio_returns)